# Dataclass

Github를 떠돌다가 dataclass라는 것을 보고 찾아보았다.

https://docs.python.org/ko/3/library/dataclasses.html

**python 3.7** 이상에서 사용가능.

In [1]:
from dataclasses import dataclass

@dataclass
class InventoryItem:
    '''Class for keeping track of an item in inventory.'''
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand

```
def __init__(self, name: str, unit_price: float, quantity_on_hand: int=0):
    self.name = name
    self.unit_price = unit_price
    self.quantity_on_hand = quantity_on_hand
``` 
는 자동으로 정의된다.

이것 외에도
 __init__() 나 __repr__() 과 같은 생성된 특수 메서드들이 자동으로 정의된다고 한다.

클래스이기 때문에 메소드도 정의할 수 있다.

[참고](https://www.python.org/dev/peps/pep-0557/)

In [9]:
knife = InventoryItem("knife", 1300, 1)

In [14]:
knife

InventoryItem(name='knife', unit_price=1300, quantity_on_hand=1)

In [11]:
knife.total_cost()

1300

In [10]:
bag = InventoryItem("bag", 124, 2)

In [13]:
bag

InventoryItem(name='bag', unit_price=124, quantity_on_hand=2)

In [12]:
bag.total_cost()

248

In [15]:
@dataclass(order=True, frozen=True)
class immutable_order:
    name: str
    price: float

order=True로 튜플처럼 대소비교가 가능하게 할수 있고, frozen=True로 불변객체로 만들 수 있다.

In [31]:
t1 = immutable_order("동주", "100")

In [17]:
t2 = immutable_order("동주", "1000")

In [23]:
t3 = immutable_order("박동주", "1000")

In [24]:
t4 = immutable_order("동주", "10000")

In [25]:
t1 > t2

False

In [26]:
t1 > t3

False

In [27]:
t2 > t3

False

In [28]:
t3 > t4

True

In [29]:
t2 > t4

False

In [32]:
t1.price = 10

FrozenInstanceError: cannot assign to field 'price'

대부분의 경우 pytorch datset과 많이 쓰이는 것 같다.

scala의 case class와 비슷한 느낌.

## 20.10.13 추가

### 불변 데이터 만들기

In [5]:
from dataclasses import dataclass
from datetime import date

@dataclass
class User:
    id: int
    name: str
    birthdate: date
    admin: bool = False

In [7]:
user1 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user1
User(id=1, name='Steve Jobs', birthdate=date(1955, 2, 24), admin=False)
user1.admin = True
user1
User(id=1, name='Steve Jobs', birthdate=date(1955, 2, 24), admin=True)

User(id=1, name='Steve Jobs', birthdate=datetime.date(1955, 2, 24), admin=True)

데이터 클래스는 기본적으로 데이터를 자유자재로 변경 가능

데이터의 불변성(immutability)가 보장되어야 하는 경우 `frozen` 옵션을 사용

In [8]:
from dataclasses import dataclass
from datetime import date


@dataclass(frozen=True)
class User:
    id: int
    name: str
    birthdate: date
    admin: bool = False

In [9]:
user1 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user1.admin = True

FrozenInstanceError: cannot assign to field 'admin'

### 데이터 대소비교 및 정렬

데이터 클래스의 인스턴스 간에 대소비교를 하려고 하면 다음과 같이 예외 발생

In [10]:
user1 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user2 = User(id=2, name="Steve Socks", birthdate=date(1955, 2, 24))

In [11]:
user1 < user2

TypeError: '<' not supported between instances of 'User' and 'User'

필드값에 따라서 데이터 대소비교가 필요하면 `order` 옵션 사용

In [12]:
from dataclasses import dataclass
from datetime import date


@dataclass(order=True)
class User:
    id: int
    name: str
    birthdate: date
    admin: bool = False

In [13]:
user1 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user2 = User(id=2, name="Bill Gates", birthdate=date(1955, 10, 28))

In [14]:
user1 < user2

True

In [15]:
user1 > user2

False

In [16]:
sorted([user2, user1])

[User(id=1, name='Steve Jobs', birthdate=datetime.date(1955, 2, 24), admin=False),
 User(id=2, name='Bill Gates', birthdate=datetime.date(1955, 10, 28), admin=False)]

### 세트나 사전에 사용하기

데이터 클래스의 인스턴스는 기본적으로 hashable 하지 않기 때문에 set의 값이나 dictionary의 값으로 사용을 하지 못함

In [17]:
set([user1, user2])

TypeError: unhashable type: 'User'

`unsafe_hash` 옵션으로 해결이 가능

In [18]:
from dataclasses import dataclass
from datetime import date


@dataclass(unsafe_hash=True)
class User:
    id: int
    name: str
    birthdate: date
    admin: bool = False

In [19]:
user1 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user2 = User(id=2, name="Bill Gates", birthdate=date(1955, 10, 28))
user3 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user4 = User(id=2, name="Bill Gates", birthdate=date(1955, 10, 28))
set([user1, user2, user3, user4])

{User(id=1, name='Steve Jobs', birthdate=datetime.date(1955, 2, 24), admin=False),
 User(id=2, name='Bill Gates', birthdate=datetime.date(1955, 10, 28), admin=False)}

### 데이터 클래스 사용시 주의사항

데이터 클래스를 사용할 때 흔히 나오는 실수는 list와 같은 가변 데이터 타입의 필드에 기본값을 할당 할 때 발생

In [20]:
from dataclasses import dataclass
from datetime import date
from typing import List


@dataclass(unsafe_hash=True)
class User:
    id: int
    name: str
    birthdate: date
    admin: bool = False
    friends: List[int] = []

ValueError: mutable default <class 'list'> for field friends is not allowed: use default_factory

dataclasses 모듈에서 제공하는 field 함수의 default_factory 옵션을 사용해서 매번 새로운 리스트가 생성될 수 있도록 해야함

In [21]:
from dataclasses import dataclass, field
from datetime import date
from typing import List


@dataclass(unsafe_hash=True)
class User:
    id: int
    name: str
    birthdate: date
    admin: bool = False
    friends: List[int] = field(default_factory=list)

In [22]:
user1 = User(id=1, name="Steve Jobs", birthdate=date(1955, 2, 24))
user1.friends
[]
user1.friends.append(2)
user1.friends

[2]

### Reference
[[파이썬] 데이터 클래스 사용법 (dataclasses 모듈)](https://www.daleseo.com/python-dataclasses/)